<a href="https://colab.research.google.com/github/citronella3alain/segment_digits/blob/main/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Installing Pyorch

!pip install torch
!pip install torchvision

In [2]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [3]:
#@title Define Hyperparameters

input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
hidden_size = 500 # number of nodes at hidden layer
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [4]:
#@title Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [18]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [6]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [7]:
net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [8]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [11]:
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images.view(-1,28*28)).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data.item()))

Epoch [1/20], Step [100/600], Loss: 0.2026
Epoch [1/20], Step [200/600], Loss: 0.2035
Epoch [1/20], Step [300/600], Loss: 0.1832
Epoch [1/20], Step [400/600], Loss: 0.2045
Epoch [1/20], Step [500/600], Loss: 0.1400
Epoch [1/20], Step [600/600], Loss: 0.0728
Epoch [2/20], Step [100/600], Loss: 0.1484
Epoch [2/20], Step [200/600], Loss: 0.0568
Epoch [2/20], Step [300/600], Loss: 0.0981
Epoch [2/20], Step [400/600], Loss: 0.0363
Epoch [2/20], Step [500/600], Loss: 0.1253
Epoch [2/20], Step [600/600], Loss: 0.0588
Epoch [3/20], Step [100/600], Loss: 0.1478
Epoch [3/20], Step [200/600], Loss: 0.0753
Epoch [3/20], Step [300/600], Loss: 0.0974
Epoch [3/20], Step [400/600], Loss: 0.0498
Epoch [3/20], Step [500/600], Loss: 0.0332
Epoch [3/20], Step [600/600], Loss: 0.1580
Epoch [4/20], Step [100/600], Loss: 0.0794
Epoch [4/20], Step [200/600], Loss: 0.0721
Epoch [4/20], Step [300/600], Loss: 0.0451
Epoch [4/20], Step [400/600], Loss: 0.0158
Epoch [4/20], Step [500/600], Loss: 0.0135
Epoch [4/20

In [12]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.390 %


In [22]:
import cv2
img3 = cv2.imread('/content/test/obj_3.png')
img4 = cv2.imread('/content/test/obj_4.png')
img3_gray = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
img4_gray = cv2.cvtColor(img4, cv2.COLOR_BGR2GRAY)

in_dat = Variable(torch.Tensor([img3_gray, img4_gray]).view(-1, 28*28)).cuda()
output = net(in_dat).cuda()
_, predicted = torch.max(output,1)
print(predicted)

tensor([9, 7], device='cuda:0')
